# **BioMistral Medical RAG Chatbot using BioMistral Open Source LLM**

## LALITH KUMAR SIRAMDASU

In [ ]:
import warnings
from pydantic import BaseModel

warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

In [ ]:
!pip install langchain sentence-transformers chromadb llama-cpp-python pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 9.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/10

In [3]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00


In [7]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 7.9 MB/s eta 0:00:00


In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

In [2]:
loder = PyPDFLoader("https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf")
docs = loder.load()

# OR  we can import the pdf from drive as
# from google.colab import drive
# drive.mount('/content/drive')
# loder = PyPDFLoader("https://drive.google.com/file/d/1JAJrxFPLBUiQXMIrVSC_nNkbm5gVaPy_/view?usp=drive_link")
# docs = loder.load()


In [3]:
len(docs)

95

In [4]:
docs[23]

Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'creationdate': '2006-02-16T11:30:29-05:00', 'subject': 'Heart disease', 'author': 'NHLBI', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:15-05:00', 'title': 'Your Guide to A Healthy Heart', 'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf', 'total_pages': 95, 'page': 23, 'page_label': '24'}, page_content='19Major Risk Factors\nGetting Tested\nHigh blood cholesterol itself does not cause symptoms, so if your cho-\nlesterol level is too high, you may not be aware of it. That’s why it is\nimportant to get your cholesterol levels checked regularly. Starting at\nage 20, everyone should have their cholesterol levels checked by\nmeans of a blood test called a lipoprotein profile. Be sure to ask for\nthe test results, so you will know whether you need to lower you

## Chunking

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks=text_splitter.split_documents(docs)

In [6]:
len(chunks)

585

In [7]:
chunks[334]

Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'creationdate': '2006-02-16T11:30:29-05:00', 'subject': 'Heart disease', 'author': 'NHLBI', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:15-05:00', 'title': 'Your Guide to A Healthy Heart', 'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf', 'total_pages': 95, 'page': 55, 'page_label': '56'}, page_content='yogurt, low-fat \nor fat-free cheese\nLean meats, poultry, 3 ounces cooked Select only lean; \nand fish lean meat, skinless trim away visible\n2 or fewer daily poultry, or fish fats; broil, roast, or\nboil, instead of \nfrying; remove \nskin from poultry\nNuts, seeds, and')

In [8]:
chunks[234]

Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'creationdate': '2006-02-16T11:30:29-05:00', 'subject': 'Heart disease', 'author': 'NHLBI', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'moddate': '2006-02-23T09:58:15-05:00', 'title': 'Your Guide to A Healthy Heart', 'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf', 'total_pages': 95, 'page': 37, 'page_label': '38'}, page_content='diabetes if your fasting blood glucose level is 126 mg/dL or higher.\nIf you have diabetes, controlling your blood glucose (blood sugar)\nlevels will help to prevent complications. Because diabetes is so\nstrongly linked with heart disease, managing diabetes must include')

## Embedding Creation

In [9]:
embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")

<ipython-input-9-0ff5cf6e8b20>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="NeuML/pubmedbert-base-embeddings")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

## Vector store creation

In [18]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 11.2 MB/s et

In [10]:
vectorstore = Chroma.from_documents(chunks,embeddings)

In [11]:
query = "Who is at risk of heart disease?"

search_results = vectorstore.similarity_search(query)

In [12]:
search_results

[Document(metadata={'creationdate': '2006-02-16T11:30:29-05:00', 'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'creator': 'QuarkXPress(tm) 6.5', 'total_pages': 95, 'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf', 'subject': 'Heart disease', 'page': 8, 'title': 'Your Guide to A Healthy Heart', 'author': 'NHLBI', 'page_label': '9', 'moddate': '2006-02-23T09:58:15-05:00', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad'}, page_content='4\nWho Is at Risk?\nRisk factors are conditions or habits that make a person more likely\nto develop a disease. They can also increase the chances that an\nexisting disease will get worse. Important risk factors for heart dis-\nease that you can do something about are cigarette smoking, high'),
 Document(metadata={'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'page_label': '9', 'total_pages': 95, 'author': 'NHLBI', 'creator': 'QuarkXPress(tm) 6.5', 

In [13]:
#Give Top 5 matches results
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [14]:
retriever.get_relevant_documents(query)

<ipython-input-14-162c3489abc9>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'total_pages': 95, 'page_label': '9', 'creator': 'QuarkXPress(tm) 6.5', 'title': 'Your Guide to A Healthy Heart', 'producer': 'Acrobat Distiller 6.0.1 for Macintosh', 'subject': 'Heart disease', 'source': 'https://www.nhlbi.nih.gov/files/docs/public/heart/healthyheart.pdf', 'creationdate': '2006-02-16T11:30:29-05:00', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad', 'page': 8, 'author': 'NHLBI', 'moddate': '2006-02-23T09:58:15-05:00'}, page_content='4\nWho Is at Risk?\nRisk factors are conditions or habits that make a person more likely\nto develop a disease. They can also increase the chances that an\nexisting disease will get worse. Important risk factors for heart dis-\nease that you can do something about are cigarette smoking, high'),
 Document(metadata={'page': 8, 'page_label': '9', 'keywords': 'heart disease, prevention, risk factors, chd, coronary artery disease, corornary heart disease, cad',

## LLM Model Loading

In [16]:
pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.7 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp311-cp311-linux_x86_64.whl size=4127096 sha256=91d9af6c4fda888d411b0a16413704fafbb2e9ad80799377a6043cfdef21615e
  Stored in directory: /root/.cache/pip/wheels/9e/8f/bf/148c8eb7d69021eccd6eae6444f3accd48347587054ffd24e5
Successfully built llama-cpp-python


In [17]:
!pip install huggingface_hub
from huggingface_hub import hf_hub_download
import os

# Download the model file
repo_id = "MaziyarPanahi/BioMistral-7B-GGUF"
filename = "BioMistral-7B.Q4_K_M.gguf"
cache_dir = "biomistral_model" # create a directory to store the model

model_path = hf_hub_download(repo_id=repo_id, filename=filename, cache_dir=cache_dir)

# Load the model using the local file path
from langchain.llms import LlamaCpp
llm = LlamaCpp(
    model_path = model_path,
    temperature = 0.2,
    max_tokens = 2048,
    top_p = 1,
)



llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from biomistral_model/models--MaziyarPanahi--BioMistral-7B-GGUF/snapshots/ae4f07544f1015dc8f5bf382b7582852638cbecf/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u3

## Use LLM and Retriever and Query to generate the final response

In [18]:
template = """
<|context|>
You are an medical assistant that follows the instructions and generate the accurate response based on query and the context provided.
please be truthful and give the answers.
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

In [19]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [20]:
# Define the prompt template
prompt = ChatPromptTemplate.from_messages([("user",template)])

In [21]:
rag_chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [22]:
response = rag_chain.invoke(query)

llama_perf_context_print:        load time =   18884.10 ms
llama_perf_context_print: prompt eval time =   18883.80 ms /    73 tokens (  258.68 ms per token,     3.87 tokens per second)
llama_perf_context_print:        eval time =   33820.89 ms /    54 runs   (  626.31 ms per token,     1.60 tokens per second)
llama_perf_context_print:       total time =   52762.81 ms /   127 tokens


In [23]:
response

'The risk of heart disease is influenced by a variety of factors, including age, family history, lifestyle, and medical history. It is generally believed that men are at higher risk than women before the age of 55 years, after which the risk increases for women.'

In [24]:
# with user input
import sys

while True:
  user_input = input(f"Enter Your Query : ")
  if user_input == "exit":
    print("Exiting query loop.")
    break
  if user_input == "":
    continue
  result = rag_chain.invoke(user_input)
  print("\nAnswer : ", result)

Enter Your Query : What are the symptoms of vitamin D deficiency?


Llama.generate: 54 prefix-match hit, remaining 22 prompt tokens to eval
llama_perf_context_print:        load time =   18884.10 ms
llama_perf_context_print: prompt eval time =    7483.10 ms /    22 tokens (  340.14 ms per token,     2.94 tokens per second)
llama_perf_context_print:        eval time =   39975.18 ms /    64 runs   (  624.61 ms per token,     1.60 tokens per second)
llama_perf_context_print:       total time =   47526.98 ms /    86 tokens



Answer :  Vitamin D deficiency can cause many health problems, including bone pain, muscle weakness, fatigue and sometimes depression. Severe vitamin D deficiency can also lead to a rare disease called rickets in children, osteomalacia in adults, or osteoporosis in both age groups.
Enter Your Query : How much water should I drink daily?


Llama.generate: 54 prefix-match hit, remaining 19 prompt tokens to eval
llama_perf_context_print:        load time =   18884.10 ms
llama_perf_context_print: prompt eval time =    4826.07 ms /    19 tokens (  254.00 ms per token,     3.94 tokens per second)
llama_perf_context_print:        eval time =   37655.65 ms /    60 runs   (  627.59 ms per token,     1.59 tokens per second)
llama_perf_context_print:       total time =   42547.32 ms /    79 tokens



Answer :  The recommended daily water intake is about 2 liters, but it varies from person to person. It depends on various factors such as age, gender, physical activity level, and environmental conditions. It is essential to drink enough water to maintain hydration levels and avoid dehydration.
Enter Your Query : exit
Exiting query loop.
